In [44]:
import os
import sys
import numpy as np
import pandas as pd
import pydicom

from glob import glob
from matplotlib import pyplot as plt
from mask_functions import rle2mask, mask2rle
from tqdm import tqdm_notebook
from tensorflow.keras import backend

In [2]:
TRAIN_PATH = './data/dicom-images-train/*/*/*.dcm'
TEST_PATH = './data/dicom-images-test/*/*/*.dcm'

In [28]:
df = pd.read_csv('./data/train-rle.csv', index_col="ImageId")

In [29]:
df.head()

,EncodedPixels
ImageId,
1.2.276.0.7230010.3.1.4.8323329.5597.1517875188.959090,-1
1.2.276.0.7230010.3.1.4.8323329.12515.1517875239.501137,-1
1.2.276.0.7230010.3.1.4.8323329.4904.1517875185.355709,175349 7 1013 12 1009 17 1005 19 1003 20 1002...
1.2.276.0.7230010.3.1.4.8323329.32579.1517875161.299312,407576 2 1021 7 1015 10 1013 12 1011 14 1008 ...
1.2.276.0.7230010.3.1.4.8323329.32579.1517875161.299312,252069 1 1021 3 1020 4 1018 5 1018 6 1016 7 1...


In [30]:
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
IMG_CHANNELS = 1

In [37]:
train_fns = glob(TRAIN_PATH)[:5000]
test_fns = glob(TEST_PATH)[:5000]

In [38]:
X_train = np.zeros((len(train_fns), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_fns), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [39]:
print("Generating training images and masks...")
sys.stdout.flush()

for n, _id in tqdm_notebook(enumerate(train_fns), total = len(train_fns)):
    print(_id)
    dataset = pydicom.read_file(_id)
    X_train[n] = np.expand_dims(dataset.pixel_array, axis=2)
    
    try:
        if '-1' in df.loc[_id.split('/')[-1][:-4], ' EncodedPixels']:
            Y_train[n] = np.zeros((1024, 1024, 1))
        else:
            if type(df.loc[_id.split('/')[-1][:-4], ' EncodedPixels']) == str:
                Y_train[n] = np.expand_dims(rle2mask(df.loc[_id.split('/')[-1][:-4], ' EncodedPixels'], 1024, 1024), axis=2)
            else:
                Y_train[n] = np.zeros((1024, 1024, 1))
                for x in df.loc[_id.split('/')[-1][:-4], ' EncodedPixels']:
                    Y_train[n] = Y_train[n] + np.expand_dims(rle2mask(x, 1024, 1024), axis=2)
    except KeyError:
        print("Key {} without mask, assuming healthy patient.".format(_id.split('/')[:1][:-4]))
        Y_train[n] = np.zeros((1024, 1024, 1)) # Assume missing mask are empty mask
        
print("Done!")
              

Generating training images and masks...


./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.1272.1517875167.67527/1.2.276.0.7230010.3.1.3.8323329.1272.1517875167.67526/1.2.276.0.7230010.3.1.4.8323329.1272.1517875167.67528.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.13352.1517875244.772591/1.2.276.0.7230010.3.1.3.8323329.13352.1517875244.772590/1.2.276.0.7230010.3.1.4.8323329.13352.1517875244.772592.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12905.1517875242.471327/1.2.276.0.7230010.3.1.3.8323329.12905.1517875242.471326/1.2.276.0.7230010.3.1.4.8323329.12905.1517875242.471328.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.14265.1517875250.487675/1.2.276.0.7230010.3.1.3.8323329.14265.1517875250.487674/1.2.276.0.7230010.3.1.4.8323329.14265.1517875250.487676.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12682.1517875241.143737/1.2.276.0.7230010.3.1.3.8323329.12682.1517875241.143736/1.2.276.0.7230010.3.1.4.8323329.12682.1517875241.143738.dcm
./data/dicom-ima

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.4485.1517875183.106810/1.2.276.0.7230010.3.1.3.8323329.4485.1517875183.106809/1.2.276.0.7230010.3.1.4.8323329.4485.1517875183.106811.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11890.1517875236.228988/1.2.276.0.7230010.3.1.3.8323329.11890.1517875236.228987/1.2.276.0.7230010.3.1.4.8323329.11890.1517875236.228989.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.2370.1517875172.391818/1.2.276.0.7230010.3.1.3.8323329.2370.1517875172.391817/1.2.276.0.7230010.3.1.4.8323329.2370.1517875172.391819.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.3197.1517875176.546325/1.2.276.0.7230010.3.1.3.8323329.3197.1517875176.546324/1.2.276.0.7230010.3.1.4.8323329.3197.1517875176.546326.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.5659.1517875189.344934/1.2.276.0.7230010.3.1.3.8323329.5659.1517875189.344933/1.2.276.0.7230010.3.1.4.8323329.5659.1517875189.344935.dcm
./data/dicom-images-tr

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12182.1517875237.857604/1.2.276.0.7230010.3.1.3.8323329.12182.1517875237.857603/1.2.276.0.7230010.3.1.4.8323329.12182.1517875237.857605.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.1759.1517875169.284900/1.2.276.0.7230010.3.1.3.8323329.1759.1517875169.284899/1.2.276.0.7230010.3.1.4.8323329.1759.1517875169.284901.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11894.1517875236.340653/1.2.276.0.7230010.3.1.3.8323329.11894.1517875236.340652/1.2.276.0.7230010.3.1.4.8323329.11894.1517875236.340654.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.4825.1517875184.1739/1.2.276.0.7230010.3.1.3.8323329.4825.1517875184.1738/1.2.276.0.7230010.3.1.4.8323329.4825.1517875184.1740.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.2586.1517875173.602856/1.2.276.0.7230010.3.1.3.8323329.2586.1517875173.602855/1.2.276.0.7230010.3.1.4.8323329.2586.1517875173.602857.dcm
./data/dicom-images-train

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.2249.1517875171.806301/1.2.276.0.7230010.3.1.3.8323329.2249.1517875171.806300/1.2.276.0.7230010.3.1.4.8323329.2249.1517875171.806302.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.13482.1517875245.521535/1.2.276.0.7230010.3.1.3.8323329.13482.1517875245.521534/1.2.276.0.7230010.3.1.4.8323329.13482.1517875245.521536.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.13690.1517875247.243989/1.2.276.0.7230010.3.1.3.8323329.13690.1517875247.243988/1.2.276.0.7230010.3.1.4.8323329.13690.1517875247.243990.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.5467.1517875188.321034/1.2.276.0.7230010.3.1.3.8323329.5467.1517875188.321033/1.2.276.0.7230010.3.1.4.8323329.5467.1517875188.321035.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12582.1517875239.890404/1.2.276.0.7230010.3.1.3.8323329.12582.1517875239.890403/1.2.276.0.7230010.3.1.4.8323329.12582.1517875239.890405.dcm
./data/dicom-ima

...

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11362.1517875232.588831/1.2.276.0.7230010.3.1.3.8323329.11362.1517875232.588830/1.2.276.0.7230010.3.1.4.8323329.11362.1517875232.588832.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.1157.1517875166.516621/1.2.276.0.7230010.3.1.3.8323329.1157.1517875166.516620/1.2.276.0.7230010.3.1.4.8323329.1157.1517875166.516622.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.10504.1517875223.885/1.2.276.0.7230010.3.1.3.8323329.10504.1517875223.884/1.2.276.0.7230010.3.1.4.8323329.10504.1517875223.886.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11774.1517875234.731489/1.2.276.0.7230010.3.1.3.8323329.11774.1517875234.731488/1.2.276.0.7230010.3.1.4.8323329.11774.1517875234.731490.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.1507.1517875168.155697/1.2.276.0.7230010.3.1.3.8323329.1507.1517875168.155696/1.2.276.0.7230010.3.1.4.8323329.1507.1517875168.155698.dcm
./data/dicom-images-train

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.2981.1517875175.538063/1.2.276.0.7230010.3.1.3.8323329.2981.1517875175.538062/1.2.276.0.7230010.3.1.4.8323329.2981.1517875175.538064.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12709.1517875241.263902/1.2.276.0.7230010.3.1.3.8323329.12709.1517875241.263901/1.2.276.0.7230010.3.1.4.8323329.12709.1517875241.263903.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.14362.1517875251.743242/1.2.276.0.7230010.3.1.3.8323329.14362.1517875251.743241/1.2.276.0.7230010.3.1.4.8323329.14362.1517875251.743243.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11409.1517875232.812813/1.2.276.0.7230010.3.1.3.8323329.11409.1517875232.812812/1.2.276.0.7230010.3.1.4.8323329.11409.1517875232.812814.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.3019.1517875175.690859/1.2.276.0.7230010.3.1.3.8323329.3019.1517875175.690858/1.2.276.0.7230010.3.1.4.8323329.3019.1517875175.690860.dcm
./data/dicom-ima

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.542.1517875163.362269/1.2.276.0.7230010.3.1.3.8323329.542.1517875163.362268/1.2.276.0.7230010.3.1.4.8323329.542.1517875163.362270.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.4558.1517875183.436315/1.2.276.0.7230010.3.1.3.8323329.4558.1517875183.436314/1.2.276.0.7230010.3.1.4.8323329.4558.1517875183.436316.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.4922.1517875185.487284/1.2.276.0.7230010.3.1.3.8323329.4922.1517875185.487283/1.2.276.0.7230010.3.1.4.8323329.4922.1517875185.487285.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11807.1517875235.117775/1.2.276.0.7230010.3.1.3.8323329.11807.1517875235.117774/1.2.276.0.7230010.3.1.4.8323329.11807.1517875235.117776.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.10052.1517875221.759904/1.2.276.0.7230010.3.1.3.8323329.10052.1517875221.759903/1.2.276.0.7230010.3.1.4.8323329.10052.1517875221.759905.dcm
./data/dicom-images-tr

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.32444.1517875160.635086/1.2.276.0.7230010.3.1.3.8323329.32444.1517875160.635085/1.2.276.0.7230010.3.1.4.8323329.32444.1517875160.635087.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.10993.1517875229.254483/1.2.276.0.7230010.3.1.3.8323329.10993.1517875229.254482/1.2.276.0.7230010.3.1.4.8323329.10993.1517875229.254484.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12009.1517875236.881189/1.2.276.0.7230010.3.1.3.8323329.12009.1517875236.881188/1.2.276.0.7230010.3.1.4.8323329.12009.1517875236.881190.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11740.1517875234.556762/1.2.276.0.7230010.3.1.3.8323329.11740.1517875234.556761/1.2.276.0.7230010.3.1.4.8323329.11740.1517875234.556763.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.12812.1517875241.969796/1.2.276.0.7230010.3.1.3.8323329.12812.1517875241.969795/1.2.276.0.7230010.3.1.4.8323329.12812.1517875241.969797.dcm
./data/dic

./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.13354.1517875244.801263/1.2.276.0.7230010.3.1.3.8323329.13354.1517875244.801262/1.2.276.0.7230010.3.1.4.8323329.13354.1517875244.801264.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.10677.1517875225.29878/1.2.276.0.7230010.3.1.3.8323329.10677.1517875225.29877/1.2.276.0.7230010.3.1.4.8323329.10677.1517875225.29879.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.11170.1517875231.513014/1.2.276.0.7230010.3.1.3.8323329.11170.1517875231.513013/1.2.276.0.7230010.3.1.4.8323329.11170.1517875231.513015.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.10451.1517875223.778478/1.2.276.0.7230010.3.1.3.8323329.10451.1517875223.778477/1.2.276.0.7230010.3.1.4.8323329.10451.1517875223.778479.dcm
./data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.5406.1517875187.956141/1.2.276.0.7230010.3.1.3.8323329.5406.1517875187.956140/1.2.276.0.7230010.3.1.4.8323329.5406.1517875187.956142.dcm
./data/dicom-ima

In [40]:
IMG_HEIGHT = 128
IMG_WIDTH = 128

In [41]:
X_train = X_train.reshape((-1, IMG_HEIGHT, IMG_WIDTH, 1))
Y_train = Y_train.reshape((-1, IMG_HEIGHT, IMG_WIDTH, 1))

In [53]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    
    return (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)

In [46]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.models import Model

In [54]:
inputs = Input((None, None, IMG_CHANNELS))

c1 = Conv2D(8, (3,3), activation='relu', padding='same')(inputs)
c1 = Conv2D(8, (3,3), activation='relu', padding='same')(c1)
p1 = MaxPooling2D((2,2))(c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (c5)
p5 = MaxPooling2D(pool_size=(2, 2)) (c5)

c55 = Conv2D(128, (3, 3), activation='relu', padding='same') (p5)
c55 = Conv2D(128, (3, 3), activation='relu', padding='same') (c55)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c55)
u6 = backend.concatenate([u6, c5])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u71 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u71 = backend.concatenate([u71, c4])
c71 = Conv2D(32, (3, 3), activation='relu', padding='same') (u71)
c61 = Conv2D(32, (3, 3), activation='relu', padding='same') (c71)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c61)
u7 = backend.concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = backend.concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = backend.concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_93 (Conv2D)              (None, None, None, 8 80          input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, None, None, 8 584         conv2d_93[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_25 (MaxPooling2D) (None, None, None, 8 0           conv2d_94[0][0]                  
____________________________________________________________________________________________

In [55]:
history = model.fit(X_train, Y_train, validation_split=.2, batch_size=512, epochs=30)

W0701 14:10:22.702141 140340348696384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 256000 samples, validate on 64000 samples
Epoch 1/30
256000/256000 [==============================] - 158s 619us/sample - loss: 0.0244 - dice_coef: 0.0070 - val_loss: 0.0170 - val_dice_coef: 0.0107
Epoch 2/30
256000/256000 [==============================] - 152s 595us/sample - loss: 0.0205 - dice_coef: 0.0091 - val_loss: 0.0161 - val_dice_coef: 0.0099
Epoch 3/30
256000/256000 [==============================] - 152s 593us/sample - loss: 0.0201 - dice_coef: 0.0103 - val_loss: 0.0160 - val_dice_coef: 0.0115
Epoch 4/30
256000/256000 [==============================] - 156s 607us/sample - loss: 0.0199 - dice_coef: 0.0115 - val_loss: 0.0158 - val_dice_coef: 0.0126
Epoch 5/30
256000/256000 [==============================] - 155s 606us/sample - loss: 0.0197 - dice_coef: 0.0125 - val_loss: 0.0160 - val_dice_coef: 0.0147
Epoch 6/30
256000/256000 [==============================] - 156s 609us/sample - loss: 0.0195 - dice_coef: 0.0140 - val_loss: 0.0160 - val_dice_coef: 0.0144
Epoch 7/30
25

In [56]:
# Show some metrics
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.subplot(2,2,2)
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('dice coefficient')
plt.ylabel('dc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')